In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.models import Model
from keras.layers import Embedding, Reshape, Dense, Activation, Dropout
from keras.models import Sequential
from keras.layers import concatenate, GlobalMaxPooling1D, Flatten, LSTM
from keras import Input
from keras.metrics import RootMeanSquaredError
from keras import backend as K

Using TensorFlow backend.


In [73]:
trainSet = pd.read_csv("C:\\Users\\krimb\\Documents\\BlackFridayDataScience\\train.csv", header=0)
testSet = pd.read_csv("C:\\Users\\krimb\\Documents\\BlackFridayDataScience\\test.csv", header=0)

trainSetLength = trainSet.shape[0]
allSet = pd.concat((trainSet, testSet), axis=0)
print(allSet.shape)
allSet.fillna(0, inplace=True)

labEncoding = ["User_ID", "Product_ID"]
for x in labEncoding:
    le = LabelEncoder()
    le.fit(allSet[x])
    allSet[x] = le.transform(allSet[x])

catEncoding = ["Gender", "Age", "Occupation", "City_Category", "Stay_In_Current_City_Years", "Marital_Status", "Product_Category_1", "Product_Category_2", "Product_Category_3"]
for y in catEncoding:
    dummies = pd.get_dummies(allSet[y])
    allSet = pd.concat((allSet, dummies), axis=1)
    allSet = allSet.drop([y], axis=1)
    print(allSet.shape)
allSet['index'] = np.arange(len(allSet))
allSet = allSet.set_index(['index'])
allSet.sort_index()
trainSet = allSet.truncate(after=(trainSetLength - 1))
testSet = allSet.truncate(before=(trainSetLength))

target = trainSet["Purchase"]
trainSet = trainSet.drop(["Purchase"], axis=1)
testSet = testSet.drop(["Purchase"], axis=1)
print(trainSet.shape)
print(testSet.shape)

(783667, 12)
(783667, 13)
(783667, 19)
(783667, 39)
(783667, 41)
(783667, 45)
(783667, 46)
(783667, 65)
(783667, 82)
(783667, 97)
(550068, 96)
(233599, 96)


In [64]:
# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
embedding1Size = pd.concat((trainSet["User_ID"], testSet["User_ID"]), axis=0).nunique()
embedding2Size = pd.concat((trainSet["Product_ID"], testSet["Product_ID"]), axis=0).nunique()
embedding1Input = Input(shape=(1,), dtype='int32', name='embedding1input')
embedding2Input = Input(shape=(1,), dtype='int32', name='embedding2input')
x = Embedding(output_dim=50, input_dim=embedding1Size, input_length=1)(embedding1Input)
y = Embedding(output_dim=50, input_dim=embedding2Size, input_length=1)(embedding2Input)
dense1Out = LSTM(32)(x)
dense2Out = LSTM(32)(y)
auxiliary_output1 = Dense(1, name='aux_output1')(dense1Out)
auxiliary_output2 = Dense(1, name='aux_output2')(dense2Out)


auxiliary_input = Input(shape=(94,), name='aux_input')
x = concatenate([dense1Out, dense2Out, auxiliary_input])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
#x = Dense(64, activation='relu')(x)
#x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(1, name='main_output')(x)

model = Model(inputs=[embedding1Input, embedding2Input, auxiliary_input], outputs=[main_output, auxiliary_output1, auxiliary_output2])

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(optimizer='rmsprop',
              loss={'main_output': root_mean_squared_error, 'aux_output1': root_mean_squared_error, 'aux_output2': root_mean_squared_error},
              loss_weights={'main_output': 1., 'aux_output1': 0.1, 'aux_output2': 0.1})

model.fit({'embedding1input': trainSet["User_ID"], 'embedding2input': trainSet["Product_ID"], 'aux_input': trainSet.drop(labEncoding, axis=1)},
          {'main_output': target, 'aux_output1': target, 'aux_output2': target, },
          epochs=50, batch_size=10000)

Epoch 1/50
550068/550068 [==============================] - 6s 11us/step - loss: 12641.0919 - main_output_loss: 10522.6611 - aux_output1_loss: 10527.0449 - aux_output2_loss: 10526.8662
Epoch 2/50
550068/550068 [==============================] - 5s 10us/step - loss: 12615.2954 - main_output_loss: 10511.3662 - aux_output1_loss: 10540.4004 - aux_output2_loss: 10539.5488
Epoch 3/50
550068/550068 [==============================] - 5s 10us/step - loss: 12558.8001 - main_output_loss: 10458.0312 - aux_output1_loss: 10541.2520 - aux_output2_loss: 10539.8193
Epoch 4/50
550068/550068 [==============================] - 5s 10us/step - loss: 12483.9927 - main_output_loss: 10392.0166 - aux_output1_loss: 10547.5850 - aux_output2_loss: 10545.9170
Epoch 5/50
550068/550068 [==============================] - 5s 10us/step - loss: 12400.5529 - main_output_loss: 10288.4590 - aux_output1_loss: 10525.6279 - aux_output2_loss: 10523.9531
Epoch 6/50
550068/550068 [==============================] - 5s 10us/step - 

In [86]:
preds = model.predict({'embedding1input': testSet["User_ID"], 'embedding2input': testSet["Product_ID"], 'aux_input': testSet.drop(labEncoding, axis=1)})[0]
print(preds)
testSetOrig = pd.read_csv("C:\\Users\\krimb\\Documents\\BlackFridayDataScience\\test.csv", header=0)
print(preds.shape)
data = {"Purchase": preds, "User_ID": testSetOrig["User_ID"]}
print(testSetOrig["User_ID"].shape)
sub = pd.DataFrame(data=data)
print(sub)

[[15957.793 ]
 [11669.232 ]
 [ 6372.971 ]
 ...
 [ 9577.84  ]
 [16306.904 ]
 [ 2939.0476]]
(233599, 1)
(233599,)


Exception: Data must be 1-dimensional